In [23]:
import os
from dotenv import load_dotenv
import openai
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationStringBufferMemory
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
from midiutil import MIDIFile
import mido

In [34]:
import mido
from mido import MidiFile, MidiTrack, Message

# Given lists
right_hand = [
    (64, 0.47), (68, 0.47), (71, 0.47), (73, 0.47), (72, 0.71), (69, 0.71), (68, 0.24), (66, 0.24), (64, 0.47), (68, 0.47), (71, 0.47), (73, 0.47), (75, 0.47), (73, 0.47), (71, 0.47), (69, 0.47), (68, 0.0), (71, 0.71), (69, 0.71), (68, 0.24), (66, 0.24), (64, 0.47), (68, 0.47), (71, 0.47), (73, 0.47)
]

left_hand = [
    (52, 0.47), (50, 0.47), (48, 0.24), (50, 0.24), (53, 0.24), (55, 0.24), (53, 0.24), (50, 0.47), (48, 0.47), (45, 0.47), (43, 0.47)
]


def create_track_from_notes(note_data, track_name=""):
    track = MidiTrack()
    track.append(mido.MetaMessage('track_name', name=track_name, time=0))
    
    # Convert the time values from the given data (in seconds) to MIDI ticks
    # Assuming a tempo of 500000 (120 BPM) and a tick rate of 480 ticks/beat
    ticks_per_second = mido.second2tick(1, ticks_per_beat=480, tempo=500000)
    
    for note, duration in note_data:
        track.append(Message('note_on', note=note, velocity=64, time=0))
        track.append(Message('note_off', note=note, velocity=64, time=int(duration * ticks_per_second)))
    
    track.append(mido.MetaMessage('end_of_track', time=0))  # Add end_of_track message
    return track

# Create a new MIDI file
mid = MidiFile()

# Add tracks for right and left hands
mid.tracks.append(create_track_from_notes(right_hand, "Right Hand"))
mid.tracks.append(create_track_from_notes(left_hand, "Left Hand"))

# Save the MIDI file
mid.save('beatles_style_melody.mid')

# Ensure you have the correct port name for GarageBand
# You can list available ports with: print(mido.get_output_names())
gb_port = "IAC Driver Bus 1"  # Replace with the correct name

# Open MIDI file and GarageBand port
mid = mido.MidiFile('beatles_style_melody.mid')
port = mido.open_output(gb_port)

# Send MIDI messages to GarageBand
for message in mid.play():
    port.send(message)

The following function read the MIDI file created and describe the
number of channels.

In [28]:
import mido
from mido import MidiFile

# Load the MIDI file
mid = MidiFile('../data/external/The_Beatles_Please_Please_Me.mid')

print(f"Number of tracks: {len(mid.tracks)}\n")

for i, track in enumerate(mid.tracks):
    print(f"Track {i + 1}: {track.name}")
    
    # Initialize time accumulator for calculating durations
    accumulated_time = 0
    note_on_time = {}  # Dictionary to store the start time of each note
    note_list = []  # List to store note and duration pairs
    
    for msg in track:
        # Convert ticks to seconds
        duration = mido.tick2second(msg.time, mid.ticks_per_beat, 500000)  # Assuming a tempo of 500000 (120 BPM)
        accumulated_time += duration
        
        if msg.type == 'note_on' and msg.velocity > 0:
            note_on_time[msg.note] = accumulated_time
        elif (msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0)) and msg.note in note_on_time:
            note_duration = accumulated_time - note_on_time[msg.note]
            note_list.append((msg.note, round(note_duration, 2)))
            del note_on_time[msg.note]  # Remove the note from the dictionary once processed
    
    # Print the note list in the desired format
    print(f"{track.name.lower().replace(' ', '_')} = [")
    for note, duration in note_list:
        print(f"    ({note}, {duration}),")
    print("]\n")



Number of tracks: 2

Track 1: 
 = [
    (64, 0.47),
    (76, 0.47),
    (63, 0.47),
    (75, 0.47),
    (61, 0.47),
    (73, 0.47),
    (59, 0.71),
    (71, 0.71),
    (61, 0.24),
    (73, 0.24),
    (59, 0.47),
    (71, 0.47),
    (56, 0.47),
    (68, 0.47),
    (64, 0.47),
    (76, 0.47),
    (63, 0.47),
    (75, 0.47),
    (61, 0.47),
    (73, 0.47),
    (59, 0.71),
    (71, 0.71),
    (61, 0.24),
    (73, 0.24),
    (59, 0.47),
    (71, 0.47),
    (56, 0.47),
    (68, 0.47),
    (76, 0.47),
    (64, 0.47),
    (68, 0.47),
    (75, 0.47),
    (76, 0.47),
    (64, 0.47),
    (68, 0.47),
    (73, 0.47),
    (76, 0.47),
    (64, 0.47),
    (68, 0.47),
    (71, 0.47),
    (76, 0.47),
    (64, 0.47),
    (68, 0.47),
    (73, 0.47),
    (76, 0.47),
    (64, 0.47),
    (68, 0.47),
    (71, 0.47),
    (76, 0.47),
    (64, 0.47),
    (68, 0.47),
    (68, 0.0),
    (76, 0.47),
    (64, 0.47),
    (68, 0.47),
    (73, 0.71),
    (76, 0.71),
    (75, 0.12),
    (64, 0.95),
    (69, 0.95),
    (